# Progama brasileiro de imunização - COVID 19

## Objetivos

Transformar os dados brutos dos registro de vacinação para um modelo analítico, disponibilizando para consulta pública.

Utilizaremos os seguintes conjuntos de dados:
* [Registros de Vacinação COVID19 do Ministério da Saúde](https://dados.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8?inner_span=True)
    * O processamento será feito por Unidade Federativa, para fins de otimização

**Os dados serão extraídos dos arquivos CSV fornecidos pelo Ministério da Saúde, e carregados em um conjunto de tabelas analíticas, seguindo o modelo [Star Schema](https://en.wikipedia.org/wiki/Star_schema).**.

## Enviroment setup

In [1]:
# Bibliotecas utilizadas
import configparser
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.types as T


In [2]:
# Read config file
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

DATA_COLUMNS = config['COMMON']['DATA_COLUMNS']
DATA_LOCATION = config['COMMON']['DATA_LOCATION']
INPUT_DATA = config[DATA_LOCATION]['INPUT_DATA']
INPUT_DATA_VACCINES = config[DATA_LOCATION]['INPUT_DATA_VACCINES']
OUTPUT_DATA = config[DATA_LOCATION]['OUTPUT_DATA']

In [3]:
def write_parquet(df, parquet_name, partitionby=None):
    parquet_path = OUTPUT_DATA + f'{parquet_name}.parquet'
    df.write.mode("overwrite").parquet(parquet_path, partitionBy=partitionby)


In [4]:
def read_parquet(parquet_name):
    parquet_path = OUTPUT_DATA + f'{parquet_name}.parquet'
    return spark.read.parquet(parquet_path)

In [5]:
def check_nulls(df, columns_list, expected_value):
    df.createOrReplaceTempView("viewcheck")
    sql_check = f"SELECT COUNT(*) FROM viewcheck WHERE 1 <> 1 {''.join([' OR ' + c + ' IS NULL ' for c in columns_list])}"
    
    dfcheck = spark.sql(sql_check)
    
    value_check = dfcheck.collect()[0][0]
    
    return value_check == expected_value

In [6]:
def check_has_content(df):
    return df.count() > 0

In [7]:
def check_uniquekey(df, columns_list):
    return df.groupBy(*columns_list).count().filter('count > 1').count() == 0

In [8]:
# Spark session
spark = SparkSession \
        .builder\
        .appName("Brasil - Programa de imunização - COVID-19")\
        .getOrCreate()

In [ ]:
dados_imunizacao_schema = T.StructType([\
    T.StructField('document_id', T.StringType(), True),\
    T.StructField('paciente_id', T.StringType(), True),\
    T.StructField('paciente_idade', T.IntegerType(), True),\
    T.StructField('paciente_datanascimento', T.DateType(), True),\
    T.StructField('paciente_enumsexobiologico', T.StringType(), True),\
    T.StructField('paciente_racacor_codigo', T.StringType(), True),\
    T.StructField('paciente_racacor_valor', T.StringType(), True),\
    T.StructField('paciente_endereco_coibgemunicipio', T.StringType(), True),\
    T.StructField('paciente_endereco_copais', T.StringType(), True),\
    T.StructField('paciente_endereco_nmmunicipio', T.StringType(), True),\
    T.StructField('paciente_endereco_nmpais', T.StringType(), True),\
    T.StructField('paciente_endereco_uf', T.StringType(), True),\
    T.StructField('paciente_endereco_cep', T.StringType(), True),\
    T.StructField('paciente_nacionalidade_enumnacionalidade', T.StringType(), True),\
    T.StructField('estabelecimento_valor', T.IntegerType(), True),\
    T.StructField('estabelecimento_razaosocial', T.StringType(), True),\
    T.StructField('estalecimento_nofantasia', T.StringType(), True),\
    T.StructField('estabelecimento_municipio_codigo', T.StringType(), True),\
    T.StructField('estabelecimento_municipio_nome', T.StringType(), True),\
    T.StructField('estabelecimento_uf', T.StringType(), True),\
    T.StructField('vacina_grupoatendimento_codigo', T.IntegerType(), True),\
    T.StructField('vacina_grupoatendimento_nome', T.StringType(), True),\
    T.StructField('vacina_categoria_codigo', T.IntegerType(), True),\
    T.StructField('vacina_categoria_nome', T.StringType(), True),\
    T.StructField('vacina_lote', T.StringType(), True),\
    T.StructField('vacina_fabricante_nome', T.StringType(), True),\
    T.StructField('vacina_fabricante_referencia', T.StringType(), True),\
    T.StructField('vacina_dataaplicacao', T.DateType(), True),\
    T.StructField('vacina_descricao_dose', T.StringType(), True),\
    T.StructField('vacina_codigo', T.IntegerType(), True),\
    T.StructField('vacina_nome', T.StringType(), True),\
    T.StructField('sistema_origem', T.StringType(), True),\
    T.StructField('data_importacao_rnds', T.StringType(), True),\
    T.StructField('id_sistema_origem', T.StringType(), True)\
])

In [ ]:
dados_imunizacao = spark.read.csv(INPUT_DATA_VACCINES, sep=';', header=True)

dados_imunizacao.printSchema()

In [ ]:
dados_imunizacao.groupBy('estabelecimento_uf').count().show()

## Step 2: Explore and Assess the Data
In this step we need:
* Explore the data to identify data quality issues, like missing values, duplicate data, etc.
* Document steps necessary to clean the data

In [ ]:
# Obter a lista de colunas que serão utilizadas
col_names = pd.read_json(DATA_COLUMNS, typ='series')
colunas_utilizadas = col_names.index
colunas_utilizadas

In [ ]:
# Obter as colunas não utilizadas
colunas_naoutilizadas = list(set(dados_imunizacao.columns) - set(colunas_utilizadas))

colunas_naoutilizadas

In [ ]:
# Excluir do dataset as colunas não utilizadas
dados_imunizacao = dados_imunizacao.drop(*colunas_naoutilizadas)
dados_imunizacao.printSchema()

In [ ]:
# Trocar os valores Nulos
dados_imunizacao = dados_imunizacao.fillna(\
    {\
        'vacina_categoria_codigo': 0, \
        'vacina_categoria_nome': 'N/A', \
        'vacina_grupoatendimento_nome': '',\
        'paciente_enumsexobiologico': 'N/A',\
        'paciente_endereco_nmmunicipio': 'N/A', \
        'paciente_endereco_nmpais': 'N/A', \
        'paciente_endereco_uf': 'N/A', \
        'estalecimento_nofantasia': 'N/A'
    })

In [ ]:
dados_imunizacao.printSchema()

In [ ]:
# Cria o arquivo parquet com os dados temporários
write_parquet(dados_imunizacao, 'dados_imunizacao', 'estabelecimento_uf')

In [9]:
dados_imunizacao = read_parquet('dados_imunizacao')
dados_imunizacao.printSchema()

root
 |-- paciente_id: string (nullable = true)
 |-- paciente_idade: string (nullable = true)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_enumsexobiologico: string (nullable = true)
 |-- paciente_racacor_valor: string (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = true)
 |-- paciente_endereco_nmpais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- estabelecimento_valor: string (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = true)
 |-- estabelecimento_municipio_nome: string (nullable = true)
 |-- vacina_grupoatendimento_codigo: string (nullable = true)
 |-- vacina_grupoatendimento_nome: string (nullable = true)
 |-- vacina_categoria_codigo: string (nullable = true)
 |-- vacina_categoria_nome: string (nullable = true)
 |-- vacina_lote: string (nullable = true)
 |-- vacina_fabricante_nome: string (nullable = true)
 |-- vacina_f

## Step 3: Define the Data Model
_Map out the conceptual data model and explain why you chose that model_

The data model is a star schema consisting of 5 Dimensions table and 1 Fact table:
  * Dimensions tables:
      * vaccines table: Vaccines and suppliers
      * health_institution table: Hospitals, Nursing home, Clinics 
      * category table: Priority groups
      * population_group table: Demograph group (professions, age group, ethnicity)
      * patient table: Demograph data (age, city, gender)
  * Fact table:
      * immunization table: Dimensions, First | second dose, date

![ER Data Model - Star Scheme](./docs/er-model-star.jpg)

_List the steps necessary to pipeline the data into the chosen data model_
* ETL starts the enviroment setup: imports, read config file, def functions and create Spark Session
* ETL script takes source data (Brazilian Government' dataset COVID-19 population immunization program)
* Raw data is read into dataframe and cleaned (remove unused columns, fill nulls) 
* For each dimension and fact table 
	* Create a temporary view table
	* Read data to new dataframe
    * Check data quality: key columns don't have nulls, each table has content
	* Create id/indexes (if necessary)
	* Write parquet files

## Step 4: Run ETL to Model the Data
* Create the data pipelines and the data model
* Include a data dictionary
* Run data quality checks to ensure the pipeline ran as expected
	* Integrity constraints on the relational database (e.g., unique key, data type, etc.)
	* Unit tests for the scripts to ensure they are doing the right thing
	* Source/count checks to ensure completeness

In [13]:
# Criar tabela Vacinas
dados_imunizacao.createOrReplaceTempView("vacinas_df")
vacinas_df = spark.sql("""
    SELECT DISTINCT CAST(vacina_codigo AS INT) AS codigo, 
                    vacina_nome AS descricao
    FROM vacinas_df
    ORDER BY codigo, descricao
""")

vacinas_df.printSchema()
vacinas_df.show()

root
 |-- codigo: integer (nullable = true)
 |-- descricao: string (nullable = true)

+------+--------------------+
|codigo|           descricao|
+------+--------------------+
|    85|Vacina Covid-19 -...|
|    86|Covid-19-Coronava...|
|    87|Vacina covid-19 -...|
|    88|Vacina covid-19 -...|
|    89|Covid-19-AstraZeneca|
+------+--------------------+



In [15]:
# Verifica valores nulos ou se há valores, antes de salvar a tabela
if not check_nulls(vacinas_df, ['codigo', 'descricao'], 0): raise Exception('Null: Vacinas')
if not check_has_content(vacinas_df): raise Exception('No content: Vacinas')
if not check_uniquekey(vacinas_df, ['codigo']): raise Exception('Unique Key Fail: Vacinas')
write_parquet(vacinas_df, 'vacinas')

In [17]:
# Criar tabela Estabelecimentos
dados_imunizacao.createOrReplaceTempView("estabelecimento_df")
estabelecimento_df = spark.sql("""
    SELECT DISTINCT CAST(estabelecimento_valor AS INT) AS codigo,
                    estalecimento_nofantasia AS descricao,
                    estabelecimento_razaosocial AS razaosocial,
                    estabelecimento_uf AS uf,
                    estabelecimento_municipio_nome AS municipio
    FROM estabelecimento_df
    ORDER BY uf, municipio, razaosocial, descricao
""")

estabelecimento_df.printSchema()
estabelecimento_df.show()

root
 |-- codigo: integer (nullable = true)
 |-- descricao: string (nullable = true)
 |-- razaosocial: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)

+-------+--------------------+--------------------+---+---------+
| codigo|           descricao|         razaosocial| uf|municipio|
+-------+--------------------+--------------------+---+---------+
|2497670|UBS DR SEBASTIAO ...|MUNICIPIO DE ABAIARA| CE|  ABAIARA|
|2497689|UBS JOAQUINA ALVE...|MUNICIPIO DE ABAIARA| CE|  ABAIARA|
|7575351|UBS MARIA DE LOUR...|MUNICIPIO DE ABAIARA| CE|  ABAIARA|
|6431208|UBS VALDEMIRO SAM...|MUNICIPIO DE ABAIARA| CE|  ABAIARA|
|2497662|UBS VILA SAO JOSE...|MUNICIPIO DE ABAIARA| CE|  ABAIARA|
|2664232|CENTRO DE SAUDE D...|      PREF M ACARAPE| CE|  ACARAPE|
|2664259|POSTO DE SAUDE DE...|      PREF M ACARAPE| CE|  ACARAPE|
|2664240|POSTO DE SAUDE SA...|      PREF M ACARAPE| CE|  ACARAPE|
|7408307|UNIDADE BASICA DE...| MINISTERIO DA SAUDE| CE|   ACARAU|
|501

In [19]:
# Verifica valores nulos ou se há valores, antes de salvar a tabela
if not check_nulls(estabelecimento_df, ['codigo', 'descricao', 'razaosocial', 'uf', 'municipio'], 0): raise Exception('Null: Estabelecimentos')
if not check_has_content(estabelecimento_df): raise Exception('No content: Estabelecimentos')
if not check_uniquekey(estabelecimento_df, ['codigo']): raise Exception('Unique Key Fail: Estabelecimentos')
write_parquet(estabelecimento_df, 'estabelecimentos')

Exception: Unique Key Fail: Estabelecimentos

In [25]:
estabelecimento_df.filter('codigo == 2464101 OR codigo == 2450712').toPandas().style

,codigo,descricao,razaosocial,uf,municipio
0,2464101,CENTRO DE SAUDE SAO CARLOS SEDE,PREFEITURA MUNICIPAL DE PALMEIRANDIA,MA,PALMEIRANDIA
1,2464101,UNIDADE MISTA SAO CARLOS,PREFEITURA MUNICIPAL DE PALMEIRANDIA,MA,PALMEIRANDIA
2,2450712,UBS FREI DIONISIO GUERRA CENTRO,PREFEITURA MUNICIPAL DE TUNTUM,MA,TUNTUM
3,2450712,UNIDADE BASICA DE SAUDE FREI DIONISIO GUERRA,PREFEITURA MUNICIPAL DE TUNTUM,MA,TUNTUM


In [ ]:
# Criar tabela Categorias
dados_imunizacao.createOrReplaceTempView("categorias_df")
categorias_df = spark.sql("""
    SELECT DISTINCT CAST(vacina_categoria_codigo AS INT) AS codigo,
                    vacina_categoria_nome AS descricao
            FROM categorias_df
            ORDER BY codigo
""")

categorias_df.printSchema()
categorias_df.show()

In [ ]:
# Verifica valores nulos ou se há valores, antes de salvar a tabela
if not check_nulls(categorias_df, ['codigo', 'descricao'], 0): raise Exception('Null: Categorias')
if not check_has_content(categorias_df): raise Exception('No content: Categorias')
if not check_uniquekey(categorias_df, ['codigo']): raise Exception('Unique Key Fail: Categorias')
write_parquet(categorias_df, 'categorias')

In [ ]:
# Criar tabela Grupos de Atendimento
dados_imunizacao.createOrReplaceTempView("grupos_atendimento_df")
grupos_atendimento_df = spark.sql("""
    SELECT DISTINCT CAST(vacina_grupoatendimento_codigo AS INT) AS codigo,
                    vacina_grupoatendimento_nome AS descricao
            FROM grupos_atendimento_df
        ORDER BY codigo
""")

grupos_atendimento_df.printSchema()
grupos_atendimento_df.show()

In [29]:
# Verifica valores nulos ou se há valores, antes de salvar a tabela
if not check_nulls(grupos_atendimento_df, ['codigo', 'descricao'], 0): raise Exception('Null: Grupos de atendimento')
if not check_has_content(grupos_atendimento_df): raise Exception('No content: Grupos de atendimento')
if not check_uniquekey(grupos_atendimento_df, ['codigo']): raise Exception('Unique Key Fail: Grupos de atendimento')
write_parquet(grupos_atendimento_df, 'grupos')

Exception: Unique Key Fail: Grupos de atendimento

In [50]:
# Create Patient table and write parquet files
dados_imunizacao.createOrReplaceTempView("vacinacao")
vacinacao_df = spark.sql("""
    SELECT DISTINCT paciente_id AS paciente_id,
            CAST(estabelecimento_valor AS INT) AS estabelecimento,
            CAST(vacina_categoria_codigo AS INT) AS categoria,
            CAST(vacina_grupoatendimento_codigo AS INT) AS grupoatendimento,
            CAST(vacina_codigo AS INT) AS vacina, 
            CAST(paciente_idade AS INT) AS idade,
            paciente_enumsexobiologico AS sexo,
            estabelecimento_uf AS uf,
            estabelecimento_municipio_nome AS municipio,
            vacina_lote AS lote,
            vacina_fabricante_nome AS fornecedor,
            vacina_descricao_dose AS dose,                  
            vacina_dataaplicacao AS dataaplicacao
        FROM vacinacao 
        WHERE paciente_id IS NOT NULL
        ORDER BY dataaplicacao
""")

vacinacao_df.printSchema()

root
 |-- paciente_id: string (nullable = true)
 |-- estabelecimento: integer (nullable = true)
 |-- categoria: integer (nullable = true)
 |-- grupoatendimento: integer (nullable = true)
 |-- vacina: integer (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- lote: string (nullable = true)
 |-- fornecedor: string (nullable = true)
 |-- dose: string (nullable = true)
 |-- dataaplicacao: string (nullable = true)



In [51]:
# Check data quality
if not check_nulls(vacinacao_df, \
        ['paciente_id', \
        'estabelecimento', \
        'categoria', \
        'grupoatendimento', \
        'vacina', \
        'idade', \
        'sexo', \
        'uf', \
        'municipio', \
        'lote', \
        'dose', \
        'dataaplicacao'], 0): raise Exception('Null: Vacinacao')
if not check_has_content(vacinacao_df): raise Exception('No content: Vacinacao')

In [52]:
# Write parquet file and get back to Spark:
write_parquet(vacinacao_df, 'vacinacao')


In [ ]:
vacinacao_df.show()

### Data dictionary

[Dict](./data-dictionary.json)
```
{
  "vaccines": {
    "description": "Vaccines and suppliers",
    "fields": {
      "id": "Identity field",
      "name": "Vaccine name",
      "supplier": "Laboratory/supplier"
    }
  },
  "health_institution": {
    "description": "Hospitals, Nursing home, Clinics",
    "fields": {
      "id": "Identity field",
      "name": "Institution name",
      "organization": "",
      "state": "Brazilian State",
      "city": "City name"
    }
  },
  "category": {
    "description": "Priority groups",
    "fields": {
      "id": "Identity field",
      "name": "Group name"
    }
  },
  "population_group": {
    "description": "Demograph group (professions, age group, ethnicity)",
    "fields": {
      "id": "Identity field",
      "name": "Group name"
    }
  },
  "patient": {
    "description": "Demograph data (age, city, gender)",
    "fields": {
      "id": "Identity field",
      "age": "Patient age (years)",
      "birthdate": "Birth date",
      "gender": "Male/Female",
      "country": "Country (Brazil)",
      "state": "Brazilian State",
      "city": "City name"
    }
  },
  "imunization": {
    "description": "Dimensions, First | second dose, date",
    "fields": {
      "patient_id": "(FK) Patient Identity",
      "health_institution_id": "(FK) Health institution Identity",
      "category_id": "(FK) Category group Identity",
      "population_group_id": "(FK) Population group Identity",
      "vaccines_id": "(FK) Vaccine Identity",
      "vaccines_dose": "First/Second dose",
      "jab_date": "Vaccine date"
    }
  }
}
```

## Step 5: Complete Project Write Up
Tools:
* Python
* Pandas
* Spark

These tools/technologies are apropriated to manipulate large dataset, processing in paralellized clusters

**ETL script should be run weekly basis**, or whenever the .gov.br update the datasets

How the script would approach the problem differently under the following scenarios:
* If the data was increased by 100x: _Use Spark Clustered to parallel the data load_
* If the pipelines were run on a daily basis by 7am: _Refactor the script to process only new informations. The [.gov.br API](https://dados.gov.br/dataset/covid-19-vacinacao/resource/97a8fbcf-941f-4d2e-91ba-dd467d5bdeac?inner_span=True) could be used to request the delta information_
* If the database needed to be accessed by 100+ people: _Store the parquet files on a cloud data lake_

### Sample queries

In [ ]:
vacinacao_df = read_parquet('vacinacao')
estabelecimento_df = read_parquet('estabelecimentos')
vacinas_df = read_parquet('vacinas')

# Obter estabelecimentos por aplicação
vacinas_df.createOrReplaceTempView("vdf")
estabelecimento_df.createOrReplaceTempView("hidf")
vacinacao_df.createOrReplaceTempView("idf")

estabelecimentos_total = spark.sql("""
    SELECT 
        CASE
            WHEN (GROUPING(hidf.descricao) = 1) THEN '# Total Estabelecimento'
            ELSE hidf.descricao
        END AS Estabelecimento,
        CASE
            WHEN (GROUPING(vdf.descricao) = 1) THEN '# Total Vacina'
            ELSE vdf.descricao
        END AS Vacina,
        COUNT(*) AS TOTAL
    FROM idf 
    INNER JOIN hidf ON idf.estabelecimento = hidf.codigo
    INNER JOIN vdf ON idf.vacina = vdf.codigo
    GROUP BY CUBE(hidf.descricao, vdf.descricao)
    ORDER BY Estabelecimento, TOTAL DESC, Vacina
""").toPandas()

estabelecimentos_total.head()